In [1]:
import pandas as pd
import numpy as np
import import_ipynb
import dividend_detective_scrape

import yfinance as yf

import datetime as dt

from datetime import datetime

importing Jupyter notebook from dividend_detective_scrape.ipynb


In [2]:
def trim_div_data(stockObj):
    actns = stockObj.actions
    for i,r in actns.iterrows():
        if r['Dividends'] > 0:
            first_div_date = i
            break

    actns = actns.loc[first_div_date:]
    return actns, first_div_date

In [3]:
def add_div_data_details(actns, first_div_date):
    actns['days_since_first_div'] = (actns.index-first_div_date).days
    actns['year'] = actns.index.year
    
    h = []
    l = []
    for i,r in actns.iterrows():
        prices = stockObj.history(start=i,end = i+pd.DateOffset(1)).iloc[0]
        h.append(prices['High'])
        l.append(prices['Low'])

    actns['Stock_High'] = h
    actns['Stock_Low'] = l
    
    actns['day_div_yield'] = actns['Dividends']/actns['Stock_High']
    return actns

In [4]:
def get_ind_div_data(stockObj):
    df, first_div_date = trim_div_data(stockObj)
    return add_div_data_details(df, first_div_date)

In [5]:
def get_div_consistency_score(stockObj):
    
    df = stockObj.actions
    df['year'] = df.index.year
    df = df[df['Dividends']>0]
    df = df.drop(columns=['Stock Splits'])
    
    ann_df = df.groupby(['year']).agg(
        {'Dividends': np.sum}).rename(columns=
        {'Dividends':'dividends'})
    start_year = ann_df.index[0]
    end_year = ann_df.index[len(ann_df)-1]+1
    ann_df = ann_df.reindex(list(range(start_year,end_year)), fill_value=0)
    
    consistancy_coeff = [1/(2**(x+1)) for x in range(len(ann_df))][::-1]
    year_progress = datetime.now().timetuple().tm_yday/365
    consistancy_coeff[-1] = consistancy_coeff[-1]*year_progress
    
    ann_df['div_consistency_score'] = consistancy_coeff
    
    highest_div = ann_df.iloc[0]['dividends']
    
#     add another weight to the last cell where i get the percentage of the year we've been in and multiply it by that

    for i,r in ann_df[1:].iterrows():      
#         i need to think about a better way to do this because right now
#         it compares the wrong two pieces
        if r['dividends'] < ann_df.loc[i-1]['dividends']:
            mult = r['div_consistency_score']
            div_perc = r['dividends']/highest_div
            ann_df.at[i, 'div_consistency_score'] = mult * div_perc
        else:
            highest_div = r['dividends']
    
    return ann_df['div_consistency_score'].sum()

In [6]:
def get_all_consistancy_scores(stockList):
    for index, row in stockList.iterrows():
        stockObj = yf.Ticker(row['ticker'])
        div_consis = get_div_consistency_score(stockObj)
        stockList.at[index, 'div_consistency_score'] = div_consis
    return stockList

In [7]:
def add_div_consistency_scores(df):
    df['div_consistency_score'] = 0.0
    df=get_all_consistancy_scores(df)
    return df

In [8]:
def get_div_stocks(min_div = 5.0, min_consistency=.5):
    df = dividend_detective_scrape.get_data()
    
    df= df[df['div_yield']>min_div]
    
    print(df)
    
    df = add_div_consistency_scores(df)   
    
    df = df[df['div_consistency_score']>min_consistency]
    df = df.sort_values('div_consistency_score', ascending=False)
    
    return df
    

In [9]:
%%time

div_stocks = get_div_stocks(min_div=6)

    ticker                       name  annual_div  div_yield
7     AGNC            AGNC Investment        1.44        8.8
16      AB  AllianceBernstein Holding        2.76        7.3
19      MO               Altria Group        3.44        7.9
27    AFIN     American Finance Trust        0.85       10.1
34     NLY  Annaly Capital Management        0.88       10.4
..     ...                        ...         ...        ...
533    VLO              Valero Energy        3.92        6.1
566    WES          Western Midstream        1.24        7.5
567   WLKP          Westlake Chemical        1.89        8.0
569    WHF         WhiteHorse Finance        1.42       10.0
571    WMB         Williams Companies        1.64        7.2

[102 rows x 4 columns]
CPU times: user 6.97 s, sys: 364 ms, total: 7.34 s
Wall time: 40.4 s


In [10]:
div_stocks

,ticker,name,annual_div,div_yield,div_consistency_score
19,MO,Altria Group,3.44,7.9,0.591543
462,STWD,Starwood Property Trust,1.92,8.9,0.591453
380,IEP,Icahn Enterprises,8.00,13.0,0.590567
445,SLRC,Solar Capital,1.64,8.7,0.588037
105,BXMT,Blackstone Mortgage Trust,2.48,8.5,0.587702
44,ARCC,Ares Capital,1.60,8.9,0.586076
569,WHF,WhiteHorse Finance,1.42,10.0,0.585850
213,DBI,Designer Brands,0.85,7.0,0.584057
362,HMLP,Hoegh LNG Partners,1.76,11.4,0.583968
102,TCPC,BlackRock TCP Capital,1.20,9.8,0.581165
